# الشبكات التوليدية

توفر الشبكات العصبية المتكررة (RNNs) ونسخها ذات الخلايا البوابية مثل خلايا الذاكرة طويلة المدى (LSTMs) ووحدات التكرار البوابية (GRUs) آلية لنمذجة اللغة، أي أنها تستطيع تعلم ترتيب الكلمات وتقديم توقعات للكلمة التالية في التسلسل. هذا يسمح لنا باستخدام الشبكات العصبية المتكررة في **المهام التوليدية**، مثل توليد النصوص العادية، الترجمة الآلية، وحتى وصف الصور.

في بنية الشبكات العصبية المتكررة التي ناقشناها في الوحدة السابقة، كل وحدة متكررة تنتج الحالة المخفية التالية كمخرج. ومع ذلك، يمكننا أيضًا إضافة مخرج آخر لكل وحدة متكررة، مما يسمح لنا بإنتاج **تسلسل** (يكون طوله مساويًا لطول التسلسل الأصلي). علاوة على ذلك، يمكننا استخدام وحدات متكررة لا تقبل مدخلًا في كل خطوة، بل تأخذ فقط متجه حالة أولية، ثم تنتج تسلسلًا من المخرجات.

في هذا الدفتر، سنركز على نماذج توليدية بسيطة تساعدنا في توليد النصوص. وللتبسيط، دعونا نبني **شبكة على مستوى الحروف**، والتي تولد النص حرفًا بحرف. أثناء التدريب، نحتاج إلى أخذ نص معين وتقسيمه إلى تسلسلات من الحروف.


In [1]:
import tensorflow as tf
from tensorflow import keras
import tensorflow_datasets as tfds
import numpy as np

ds_train, ds_test = tfds.load('ag_news_subset').values()

## بناء مفردات الحروف

لبناء شبكة توليد على مستوى الحروف، نحتاج إلى تقسيم النص إلى حروف فردية بدلاً من الكلمات. طبقة `TextVectorization` التي كنا نستخدمها سابقاً لا يمكنها القيام بذلك، لذا لدينا خياران:

* تحميل النص يدوياً وتنفيذ عملية التقطيع "يدوياً"، كما هو موضح في [هذا المثال الرسمي من Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/)
* استخدام فئة `Tokenizer` للتقطيع على مستوى الحروف.

سنختار الخيار الثاني. يمكن أيضاً استخدام `Tokenizer` للتقطيع إلى كلمات، لذا يمكن التبديل بسهولة من التقطيع على مستوى الحروف إلى التقطيع على مستوى الكلمات.

لإجراء التقطيع على مستوى الحروف، نحتاج إلى تمرير المعامل `char_level=True`:


In [2]:
def extract_text(x):
    return x['title']+' '+x['description']

def tupelize(x):
    return (extract_text(x),x['label'])

tokenizer = keras.preprocessing.text.Tokenizer(char_level=True,lower=False)
tokenizer.fit_on_texts([x['title'].numpy().decode('utf-8') for x in ds_train])

نريد أيضًا استخدام رمز خاص للإشارة إلى **نهاية التسلسل**، والذي سنسميه `<eos>`. دعنا نضيفه يدويًا إلى المفردات:


In [3]:
eos_token = len(tokenizer.word_index)+1
tokenizer.word_index['<eos>'] = eos_token

vocab_size = eos_token + 1

الآن، لترميز النص إلى تسلسلات من الأرقام، يمكننا استخدام:


In [4]:
tokenizer.texts_to_sequences(['Hello, world!'])

[[48, 2, 10, 10, 5, 44, 1, 25, 5, 8, 10, 13, 78]]

## تدريب شبكة عصبية متكررة (RNN) لتوليد العناوين

الطريقة التي سنقوم بها بتدريب الشبكة العصبية المتكررة لتوليد عناوين الأخبار هي كالتالي. في كل خطوة، سنأخذ عنوانًا واحدًا، يتم إدخاله في الشبكة العصبية المتكررة، وبالنسبة لكل حرف مُدخل، سنطلب من الشبكة توليد الحرف التالي:

![صورة توضح مثالًا لتوليد كلمة "HELLO" باستخدام شبكة عصبية متكررة.](../../../../../translated_images/ar/rnn-generate.56c54afb52f9781d.webp)

بالنسبة للحرف الأخير في تسلسلنا، سنطلب من الشبكة توليد رمز `<eos>`.

الفرق الرئيسي بين الشبكة العصبية المتكررة التوليدية التي نستخدمها هنا هو أننا سنأخذ الناتج من كل خطوة من الشبكة العصبية المتكررة، وليس فقط من الخلية النهائية. يمكن تحقيق ذلك عن طريق تحديد معامل `return_sequences` في خلية الشبكة العصبية المتكررة.

لذلك، أثناء التدريب، سيكون الإدخال إلى الشبكة عبارة عن تسلسل من الأحرف المشفرة بطول معين، والناتج سيكون تسلسلًا بنفس الطول، ولكن مُزاحًا بعنصر واحد وينتهي برمز `<eos>`. ستتكون الدفعة الصغيرة (Minibatch) من عدة تسلسلات مثل هذه، وسنحتاج إلى استخدام **التعبئة** لتنسيق جميع التسلسلات.

لنقم بإنشاء وظائف لتحويل مجموعة البيانات لنا. نظرًا لأننا نريد تعبئة التسلسلات على مستوى الدفعة الصغيرة، سنقوم أولاً بتجميع مجموعة البيانات عن طريق استدعاء `.batch()`، ثم استخدام `map` لإجراء التحويل. وبالتالي، ستأخذ وظيفة التحويل دفعة صغيرة كاملة كمعامل:


In [5]:
def title_batch(x):
    x = [t.numpy().decode('utf-8') for t in x]
    z = tokenizer.texts_to_sequences(x)
    z = tf.keras.preprocessing.sequence.pad_sequences(z)
    return tf.one_hot(z,vocab_size), tf.one_hot(tf.concat([z[:,1:],tf.constant(eos_token,shape=(len(z),1))],axis=1),vocab_size)

بعض الأمور المهمة التي نقوم بها هنا:
* أولاً، نستخرج النص الفعلي من المصفوفة النصية
* `text_to_sequences` يحول قائمة النصوص إلى قائمة من مصفوفات الأعداد الصحيحة
* `pad_sequences` يقوم بتعبئة تلك المصفوفات إلى أقصى طول لها
* وأخيراً، نقوم بترميز جميع الأحرف باستخدام الترميز الواحد، بالإضافة إلى القيام بالإزاحة وإضافة `<eos>`. سنرى قريباً لماذا نحتاج إلى الأحرف المرمزة بالترميز الواحد

ومع ذلك، هذه الوظيفة **Pythonic**، أي أنها لا يمكن ترجمتها تلقائياً إلى الرسم البياني الحسابي الخاص بـ Tensorflow. سنواجه أخطاء إذا حاولنا استخدام هذه الوظيفة مباشرة في وظيفة `Dataset.map`. نحتاج إلى تضمين هذا الاستدعاء Pythonic باستخدام غلاف `py_function`:


In [6]:
def title_batch_fn(x):
    x = x['title']
    a,b = tf.py_function(title_batch,inp=[x],Tout=(tf.float32,tf.float32))
    return a,b

> **ملاحظة**: قد يبدو التمييز بين وظائف التحويل في Python و Tensorflow معقدًا بعض الشيء، وربما تتساءل لماذا لا نقوم بتحويل مجموعة البيانات باستخدام وظائف Python القياسية قبل تمريرها إلى `fit`. بينما يمكن بالتأكيد القيام بذلك، فإن استخدام `Dataset.map` له ميزة كبيرة، حيث يتم تنفيذ خط أنابيب تحويل البيانات باستخدام الرسم البياني الحسابي لـ Tensorflow، مما يستفيد من حسابات GPU ويقلل الحاجة إلى تمرير البيانات بين CPU و GPU.

الآن يمكننا بناء شبكة المولد الخاصة بنا والبدء في التدريب. يمكن أن تعتمد على أي خلية متكررة ناقشناها في الوحدة السابقة (بسيطة، LSTM أو GRU). في مثالنا سنستخدم LSTM.

نظرًا لأن الشبكة تأخذ الأحرف كمدخلات وحجم المفردات صغير جدًا، فلا نحتاج إلى طبقة تضمين، حيث يمكن إدخال المدخلات المشفرة بطريقة "one-hot" مباشرةً إلى خلية LSTM. ستكون طبقة الإخراج عبارة عن مصنف `Dense` يحول مخرجات LSTM إلى أرقام الرموز المشفرة بطريقة "one-hot".

بالإضافة إلى ذلك، نظرًا لأننا نتعامل مع تسلسلات ذات أطوال متغيرة، يمكننا استخدام طبقة `Masking` لإنشاء قناع يتجاهل الجزء المعبأ من النص. هذا ليس ضروريًا تمامًا، لأننا لسنا مهتمين كثيرًا بكل ما يتجاوز رمز `<eos>`، ولكننا سنستخدمه من أجل اكتساب بعض الخبرة مع هذا النوع من الطبقات. سيكون `input_shape` هو `(None, vocab_size)`، حيث يشير `None` إلى تسلسل بطول متغير، وشكل الإخراج هو `(None, vocab_size)` أيضًا، كما يمكنك أن ترى من `summary`:


In [7]:
model = keras.models.Sequential([
    keras.layers.Masking(input_shape=(None,vocab_size)),
    keras.layers.LSTM(128,return_sequences=True),
    keras.layers.Dense(vocab_size,activation='softmax')
])

model.summary()
model.compile(loss='categorical_crossentropy')

model.fit(ds_train.batch(8).map(title_batch_fn))

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
masking (Masking)            (None, None, 84)          0         
_________________________________________________________________
lstm (LSTM)                  (None, None, 128)         109056    
_________________________________________________________________
dense (Dense)                (None, None, 84)          10836     
Total params: 119,892
Trainable params: 119,892
Non-trainable params: 0
_________________________________________________________________
15000/15000 [==============================] - 229s 15ms/step - loss: 1.5385


## إنشاء المخرجات

الآن بعد أن قمنا بتدريب النموذج، نريد استخدامه لتوليد بعض المخرجات. أولاً، نحتاج إلى طريقة لفك تشفير النص الممثل بواسطة سلسلة من أرقام الرموز. للقيام بذلك، يمكننا استخدام وظيفة `tokenizer.sequences_to_texts`؛ ومع ذلك، فهي لا تعمل بشكل جيد مع الترميز على مستوى الحروف. لذلك سنأخذ قاموس الرموز من الـ `tokenizer` (يسمى `word_index`)، ونبني خريطة عكسية، ونكتب وظيفة فك تشفير خاصة بنا:


In [10]:
reverse_map = {val:key for key, val in tokenizer.word_index.items()}

def decode(x):
    return ''.join([reverse_map[t] for t in x])

الآن، دعونا نبدأ عملية التوليد. سنبدأ بسلسلة نصية `start`، ونقوم بترميزها إلى تسلسل `inp`. بعد ذلك، في كل خطوة، سنستدعي الشبكة الخاصة بنا لاستنتاج الحرف التالي.

الناتج من الشبكة `out` هو متجه يحتوي على `vocab_size` عناصر تمثل احتمالات كل رمز، ويمكننا العثور على الرقم الأكثر احتمالاً للرمز باستخدام `argmax`. ثم نقوم بإضافة هذا الحرف إلى القائمة المولدة من الرموز، ونستمر في عملية التوليد. يتم تكرار هذه العملية لتوليد حرف واحد `size` مرة لتوليد العدد المطلوب من الأحرف، ونتوقف مبكرًا عندما يتم الوصول إلى `eos_token`.


In [12]:
def generate(model,size=100,start='Today '):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            nc = tf.argmax(out)
            if nc==eos_token:
                break
            chars.append(nc.numpy())
            inp = inp+[nc]
        return decode(chars)
    
generate(model)

'Today #39;s lead to strike for the strike for the strike for the strike (AFP)'

## أخذ عينات من المخرجات أثناء التدريب

نظرًا لعدم وجود أي مقاييس مفيدة مثل *الدقة*، فإن الطريقة الوحيدة التي يمكننا من خلالها معرفة أن النموذج يتحسن هي من خلال **أخذ عينات** من النصوص المُولدة أثناء التدريب. للقيام بذلك، سنستخدم **الوظائف التلقائية**، أي الوظائف التي يمكننا تمريرها إلى دالة `fit`، والتي سيتم استدعاؤها بشكل دوري أثناء التدريب.


In [13]:
sampling_callback = keras.callbacks.LambdaCallback(
  on_epoch_end = lambda batch, logs: print(generate(model))
)

model.fit(ds_train.batch(8).map(title_batch_fn),callbacks=[sampling_callback],epochs=3)

Epoch 1/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.2703
Today #39;s a lead in the company for the strike
Epoch 2/3
15000/15000 [==============================] - 227s 15ms/step - loss: 1.2057
Today #39;s the Market Service on Security Start (AP)
Epoch 3/3
15000/15000 [==============================] - 226s 15ms/step - loss: 1.1752
Today #39;s a line on the strike to start for the start


هذا المثال يولد نصًا جيدًا بالفعل، ولكنه يمكن تحسينه بعدة طرق:

* **إضافة المزيد من النصوص**. لقد استخدمنا فقط العناوين لمهمتنا، ولكن قد ترغب في تجربة النصوص الكاملة. تذكر أن الشبكات العصبية التكرارية (RNNs) ليست جيدة جدًا في التعامل مع التسلسلات الطويلة، لذا من المنطقي إما تقسيمها إلى جمل أقصر، أو دائمًا التدريب على طول تسلسل ثابت بقيمة محددة مسبقًا `num_chars` (على سبيل المثال، 256). يمكنك محاولة تعديل المثال أعلاه إلى مثل هذه البنية باستخدام [البرنامج التعليمي الرسمي لـ Keras](https://keras.io/examples/generative/lstm_character_level_text_generation/) كمصدر إلهام.

* **استخدام LSTM متعدد الطبقات**. من المنطقي تجربة 2 أو 3 طبقات من خلايا LSTM. كما ذكرنا في الوحدة السابقة، كل طبقة من LSTM تستخرج أنماطًا معينة من النص، وفي حالة المولد على مستوى الحروف يمكننا توقع أن تكون الطبقة الأدنى مسؤولة عن استخراج المقاطع الصوتية، والطبقات الأعلى - عن الكلمات وتركيبات الكلمات. يمكن تنفيذ ذلك ببساطة عن طريق تمرير معلمة عدد الطبقات إلى منشئ LSTM.

* قد ترغب أيضًا في تجربة **وحدات GRU** لمعرفة أيها يقدم أداءً أفضل، وتجربة **أحجام مختلفة للطبقات المخفية**. الطبقة المخفية الكبيرة جدًا قد تؤدي إلى الإفراط في التكيف (على سبيل المثال، ستتعلم الشبكة النص بشكل دقيق جدًا)، بينما الحجم الأصغر قد لا ينتج نتائج جيدة.


## توليد النص الناعم ودرجة الحرارة

في التعريف السابق لـ `generate`، كنا دائمًا نأخذ الحرف ذو أعلى احتمال ليكون الحرف التالي في النص المُولد. أدى ذلك إلى أن النص غالبًا ما "يدور" بين نفس تسلسلات الأحرف مرارًا وتكرارًا، كما في هذا المثال:
```
today of the second the company and a second the company ...
```

ومع ذلك، إذا نظرنا إلى توزيع الاحتمالات للحرف التالي، قد نجد أن الفرق بين أعلى الاحتمالات ليس كبيرًا، على سبيل المثال، يمكن أن يكون احتمال أحد الأحرف 0.2، وآخر 0.19، وهكذا. على سبيل المثال، عند البحث عن الحرف التالي في التسلسل '*play*'، يمكن أن يكون الحرف التالي بنفس الاحتمال إما مسافة أو **e** (كما في كلمة *player*).

هذا يقودنا إلى استنتاج أنه ليس من "العدل" دائمًا اختيار الحرف ذو الاحتمال الأعلى، لأن اختيار الحرف الثاني الأعلى قد يؤدي أيضًا إلى نص ذو معنى. من الأكثر حكمة أن نقوم بـ **أخذ عينات** من الأحرف بناءً على توزيع الاحتمالات الناتج عن الشبكة.

يمكن تنفيذ هذه العملية باستخدام دالة `np.multinomial` التي تطبق ما يُعرف بـ **التوزيع متعدد الحدود**. الدالة التي تنفذ هذا التوليد النصي **الناعم** مُعرفة أدناه:


In [33]:
def generate_soft(model,size=100,start='Today ',temperature=1.0):
        inp = tokenizer.texts_to_sequences([start])[0]
        chars = inp
        for i in range(size):
            out = model(tf.expand_dims(tf.one_hot(inp,vocab_size),0))[0][-1]
            probs = tf.exp(tf.math.log(out)/temperature).numpy().astype(np.float64)
            probs = probs/np.sum(probs)
            nc = np.argmax(np.random.multinomial(1,probs,1))
            if nc==eos_token:
                break
            chars.append(nc)
            inp = inp+[nc]
        return decode(chars)

words = ['Today ','On Sunday ','Moscow, ','President ','Little red riding hood ']
    
for i in [0.3,0.8,1.0,1.3,1.8]:
    print(f"\n--- Temperature = {i}")
    for j in range(5):
        print(generate_soft(model,size=300,start=words[j],temperature=i))


--- Temperature = 0.3
Today #39;s strike #39; to start at the store return
On Sunday PO to Be Data Profit Up (Reuters)
Moscow, SP wins straight to the Microsoft #39;s control of the space start
President olding of the blast start for the strike to pay &lt;b&gt;...&lt;/b&gt;
Little red riding hood ficed to the spam countered in European &lt;b&gt;...&lt;/b&gt;

--- Temperature = 0.8
Today countie strikes ryder missile faces food market blut
On Sunday collores lose-toppy of sale of Bullment in &lt;b&gt;...&lt;/b&gt;
Moscow, IBM Diffeiting in Afghan Software Hotels (Reuters)
President Ol Luster for Profit Peaced Raised (AP)
Little red riding hood dace on depart talks #39; bank up

--- Temperature = 1.0
Today wits House buiting debate fixes #39; supervice stake again
On Sunday arling digital poaching In for level
Moscow, DS Up 7, Top Proble Protest Caprey Mamarian Strike
President teps help of roubler stepted lessabul-Dhalitics (AFP)
Little red riding hood signs on cash in Carter-youb

---

KeyError: 0

لقد قدمنا ​​معلمة إضافية تسمى **درجة الحرارة**، والتي تُستخدم للإشارة إلى مدى التزامنا بأعلى احتمال. إذا كانت درجة الحرارة 1.0، نقوم بأخذ عينات متعددة بشكل عادل، وعندما تقترب درجة الحرارة من اللانهاية - تصبح جميع الاحتمالات متساوية، ونختار الحرف التالي عشوائيًا. في المثال أدناه يمكننا ملاحظة أن النص يصبح بلا معنى عندما نزيد درجة الحرارة كثيرًا، ويشبه النص "المتكرر" الذي يتم إنشاؤه بصعوبة عندما تقترب من 0.



---

**إخلاء المسؤولية**:  
تم ترجمة هذا المستند باستخدام خدمة الترجمة بالذكاء الاصطناعي [Co-op Translator](https://github.com/Azure/co-op-translator). بينما نسعى لتحقيق الدقة، يرجى العلم أن الترجمات الآلية قد تحتوي على أخطاء أو معلومات غير دقيقة. يجب اعتبار المستند الأصلي بلغته الأصلية المصدر الموثوق. للحصول على معلومات حاسمة، يُوصى بالاستعانة بترجمة بشرية احترافية. نحن غير مسؤولين عن أي سوء فهم أو تفسيرات خاطئة ناتجة عن استخدام هذه الترجمة.
